<a href="https://colab.research.google.com/github/shizoda/education/blob/main/machine_learning/optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 最適化器（オプティマイザ）

**最適化器（オプティマイザ）** とは、学習率を調節したり、どのような方向にネットワーク内の重みを修正するかを自動的に調節するアルゴリズムのことです。モデルの性能を最大限に引き出すためには、適切なオプティマイザを選択し、その動作をうまく調整することが重要です。

機械学習において **初期学習率 (learning rate)** を適切に設定することは、学習プロセスの効率を決める重要な要素です。学習率が大きすぎると、損失関数の最小値に到達する前に値が発散したり、過学習を引き起こすことがあります。一方で、学習率が小さすぎると、モデルが収束するのに長い時間がかかります。

また、学習が進むにつれて、学習率を動的に調節する必要があります。学習の初期段階では、モデルがまだ正しい方向性を見つけられていないため、大きな学習率を設定して、大まかな方向を素早く探ります。しかし、学習が進んで損失関数が安定してくる段階では、学習率を小さくして慎重に調整することが重要です。このようにすることで、最適解に近づく際に余計な振動や遠ざかりを防ぐことができます。このような調節をスケジューリングといい、最初は大きめの値として、後半では徐々に小さくすることが多いです。


### そもそも最小化とは

機械学習の目的は、モデルが誤差（損失関数）を最小限に抑えることです。損失関数とは、モデルがどれだけ正しい予測をしているかを数値で示したものです。例えば、予測と実際の値が大きく離れているほど損失関数の値は大きくなり、逆に一致しているほど小さくなります。

この損失関数を最小化するためには、数学的には関数の最小値を求める「最適化問題」を解く必要があります。ここで着目すべきなのが **勾配（関数の傾き）**です。

### 最小化の手順

基本的な **勾配降下法 (Gradient Descent)** という方法で、関数が最小となるパラメータを探してみましょう。

- 勾配の計算

損失関数を入力パラメータについて微分して、**現時点からどの方向に進めば関数の値が減るか** を調べます。

- パラメータの更新

勾配が示す方向とは逆の方向に少しだけ進むことで、損失を少しずつ減らします。この際、更新の幅を決めるのが **学習率（learning rate） ** です。

- 繰り返し

この操作を何度も繰り返して、損失が最小になるパラメータを探します。今回はステップごと  ``decay_rate`` 0.9 倍で学習率を減らしていくことにします。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange

# 初期値と学習率
x = 0  # 初期点
initial_learning_rate = 0.01
decay_rate = 0.9  # 減衰率

# 最大ステップ数
steps = 20

# 損失関数とその勾配を定義
def loss_function(x):
    return x**4 - 3*x**3 - 7*x**2 + 5*x  # 損失関数

def gradient(x):
    return 4*x**3 - 9*x**2 - 8*x + 5  # 損失関数の微分

# 記録用リスト
path = [x]
learning_rates = []  # 学習率の変化を記録

# 最適化
for step in trange(steps):
    current_learning_rate = initial_learning_rate * (decay_rate ** step)  # 学習率の減少
    learning_rates.append(current_learning_rate)
    grad = gradient(x)  # 勾配の計算
    x -= current_learning_rate * grad  # パラメータの更新
    path.append(x)

# 可視化
x_vals = np.linspace(-3, 5, 500)
y_vals = loss_function(x_vals)

plt.figure(figsize=(12, 5))

# 損失関数と最適化経路
plt.subplot(1, 2, 1)
plt.plot(x_vals, y_vals, label="Loss Function", color="blue")
plt.scatter(path, [loss_function(x) for x in path], color="red", label="SGD Steps")
plt.title("Gradient Descent with Learning Rate Decay")
plt.xlabel("x")
plt.ylabel("y")
plt.axhline(0, color="black", linestyle="--", linewidth=0.5)
plt.axvline(0, color="black", linestyle="--", linewidth=0.5)
plt.legend()
plt.grid()

# 学習率の変化
plt.subplot(1, 2, 2)
plt.plot(range(steps), learning_rates, label="Learning Rate", marker="o", color="red")
plt.title("Learning Rate Decay")
plt.xlabel("Steps")
plt.ylabel("Learning Rate")
plt.grid()
plt.legend()

plt.tight_layout()
plt.show()


## 演習１

初期値や学習率を調節して、大域解に到達するようにしてください。

また、なるべく高速に（少ない回数で）到達できるようにしてください。初期値が適切でも、あまりにも学習率を上げすぎるとうまくいかないことも確認してください。

----


### **確率的勾配降下法（SGD）**

上記の勾配降下法 (GD) には以下の課題があります：
- データ全体（全データセット）を使って勾配を計算するため、計算コストが高い。
- 学習率の調整が難しく、適切な収束が保証されない場合がある。

これらの問題を解決するために登場したのが **確率的勾配降下法（SGD）** です。SGDは勾配降下法を改良した手法で、以下の特徴があります：

1. **ミニバッチの使用**:  
   全データセットの代わりに、ランダムに選ばれたデータの小さな部分集合（ミニバッチ）を使って勾配を計算します。
2. **計算コストの削減**:  
   一部のデータだけを使うため、1回の更新の計算が効率的です。
3. **不確実性（ノイズ）の活用**:  
   各更新にランダム性が含まれるため、**局所解（local minimum）**から脱出しやすいです。

| 手法                | 使用するデータ              | 更新回数            | 特徴                              |
|---------------------|---------------------------|---------------------|-----------------------------------|
| 勾配降下法（GD）    | 全データ                  | 1エポック1回更新    | 安定だが計算コストが高い           |
| 確率的勾配降下法（SGD） | ミニバッチ                | データ数 ÷ バッチサイズ回 | ノイズを含むが計算が効率的         |
| SGD + 学習率減衰     | ミニバッチ                | 同上                | 初期は探索、後半は安定した収束     |
| SGD + Momentum       | ミニバッチ                | 同上                | 更新がスムーズで振動が抑えられる   |



### **SGDの更新式**
GDの更新式は次のように表されます：

$$
\theta_{t+1} = \theta_t - \eta \nabla L(\theta_t)
$$

一方、SGDでは、損失関数 $L(\theta_t)$ を全データではなくミニバッチ $B$ で近似するため、次のように更新します：

$$
\theta_{t+1} = \theta_t - \eta \nabla L_B(\theta_t)
$$

ここで：
- $B$: ミニバッチのデータ
- $L_B(\theta_t)$: ミニバッチで計算された損失関数


### **学習率減衰（Learning Rate Decay）**
学習率減衰は、SGDにも適用可能であり、以下の理由で重要です：

1. **学習初期の探索性**: 初期段階では大きなステップサイズで広範囲を探索します。
2. **収束時の安定性**: 最適化が進むにつれてステップサイズを小さくすることで、最終的な解への収束を安定させます。

学習率減衰は、たとえば指数関数的減衰（Exponential Decay）で以下のように書けます：
$$
\eta_t = \eta_0 r^t
$$

ここで：
- $\eta_t$: ステップ $t$ における学習率
- $\eta_0$: 初期学習率
- $r$: 減衰率（例えば 0.9）

### **SGDとMomentumの組み合わせ**
SGD単独では、不安定な振動が起こる場合があります。これを改善するために、**Momentum** を加えることが一般的です。Momentumは、過去の更新方向の情報を利用して、更新をスムーズにする手法です。

#### **Momentumの更新式**
$$
v_t = \beta v_{t-1} - \eta \nabla L_B(\theta_t)
$$
$$
\theta_{t+1} = \theta_t + v_t
$$

ここで：
- $v_t$: 過去の更新量の累積
- $\beta$: Momentum係数（通常は 0.9 など）
- $\eta$: 学習率
- $\nabla L_B(\theta_t)$: ミニバッチ $B$ における損失関数の勾配



### 演習２

SGD を用いて iris の分類問題を実行し、どのようなスピードで学習率が低下していくのか確認してみましょう。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from logging import critical
from torch.optim.lr_scheduler import ExponentialLR


# Iris データセットの読み込み
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Virginica とそれ以外の2クラスに限定
y = (y == 2).astype(int)  # Virginica: 1, それ以外: 0

# 特徴量を2次元に限定（Sepal Length, Sepal Width）
X = X[:, :2]

# データの標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)

# データをトレーニングセットとテストセットに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# サンプル点の可視化
plt.figure(figsize=(8, 6))
plt.scatter(X_train[y_train == 0, 0], X_train[y_train == 0, 1], c="green", marker="o", label="Train - Not Versicolor")
plt.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], c="blue", marker="o", label="Train - Versicolor")
plt.scatter(X_test[y_test == 0, 0], X_test[y_test == 0, 1], c="green", marker="x", label="Test - Not Versicolor")
plt.scatter(X_test[y_test == 1, 0], X_test[y_test == 1, 1], c="blue", marker="x", label="Test - Versicolor")
plt.title("Train/Test Split Visualization (Versicolor vs Others)")
plt.xlabel("Feature 1 (Standardized)")
plt.ylabel("Feature 2 (Standardized)")
plt.legend()
plt.grid()
plt.show()


In [ ]:
# 学習率減衰を適用する最適化関数
def sgd_with_exp_decay(momentum=1.0):
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=momentum)
    scheduler = ExponentialLR(optimizer, gamma=0.9)  # 学習率を指数関数的に減少
    return optimizer, scheduler

# オプティマイザのリスト
optimizers = {
    "SGD": lambda: optim.SGD(model.parameters(), lr=0.1),
    "SGD_with_decay":  lambda: sgd_with_exp_decay(momentum=1.0),
    # "SGD_with_decay_and_momentum": lambda:
    # 他にも 「"名前": lambda: 呼び出し文」 という書き方で追加してください
}


# シンプルなMLPモデル
model = nn.Sequential(
    nn.Linear(2, 3),  # 特徴量数: 2, 出力: 3（3クラス分類）
    nn.Softmax(dim=1)  # 出力を確率分布に変換
)

criterion = nn.CrossEntropyLoss()  # クロスエントロピー損失を定義

In [ ]:
# トレーニング関数
def train(optimizer_name):
    if type(optimizers[optimizer_name]()) in [tuple, list]:
        optimizer, scheduler = optimizers[optimizer_name]()
    else:
        optimizer = optimizers[optimizer_name]()
        scheduler = None

    losses = []  # ロスを記録するリスト
    lrs = []     # 学習率を記録するリスト

    for epoch in range(300):
        optimizer.zero_grad()
        y_pred = model(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor)
        loss.backward()
        optimizer.step()

        if scheduler:
            scheduler.step()  # 学習率を更新

        # 各エポックでのロスと学習率を記録
        losses.append(loss.item())
        lrs.append(optimizer.param_groups[0]['lr'])

    return losses, lrs  # ロスと学習率のリストを返す


# トレーニングと可視化
for optimizer_name in optimizers.keys():
    model.apply(lambda m: m.reset_parameters() if hasattr(m, "reset_parameters") else None)  # 重み初期化
    losses, lrs = train(optimizer_name)  # 修正後の train 関数

    # 横並びのグラフ作成
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    # 学習曲線と学習率の変化を左側にプロット
    ax1 = axes[0]
    ax2 = ax1.twinx()
    ax1.plot(losses, label="Loss", color="blue")
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss", color="blue")
    ax1.tick_params(axis='y', labelcolor="blue")
    ax2.plot(lrs, label="Learning Rate", color="red")
    ax2.set_ylabel("Learning Rate", color="red")
    ax2.tick_params(axis='y', labelcolor="red")
    ax1.set_title(f"Loss and Learning Rate ({optimizer_name})")

    # 決定境界を右側にプロット
    ax3 = axes[1]
    xx, yy = np.meshgrid(np.linspace(-3, 3, 100), np.linspace(-3, 3, 100))
    grid = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

    # Get the class with the maximum probability for each grid point
    preds = model(grid).detach().numpy()
    # predicted_classes = np.argmax(preds, axis=1)
    predicted_classes = preds[:, 1]

    # Reshape the predictions to match the grid shape
    predicted_classes = predicted_classes.reshape(xx.shape)


    # Plot the decision boundary
    ax3.contourf(xx, yy, predicted_classes, levels=50, cmap="RdBu", alpha=0.3)
    ax3.scatter(X_train[y_train == 0, 0], X_train[y_train == 0, 1], c="red", marker="o", label="Not Virginica")
    ax3.scatter(X_train[y_train == 1, 0], X_train[y_train == 1, 1], c="blue", marker="s", label="Virginica")
    ax3.set_title(f"Decision Boundary ({optimizer_name})")
    ax3.legend()

    ax3.legend()

    # レイアウト調整して表示
    fig.tight_layout()
    plt.show()


### 演習３

SGD 以外にも、いくつか有名なオプティマイザがあります。optimizers に追加して実行し、SGD との速度や結果の違いを見てみましょう。

| オプティマイザ名   | 特徴                                                                 | PyTorchの関数名                |
|--------------------|----------------------------------------------------------------------|--------------------------------|
| SGD               | 確率的勾配降下法。シンプルで計算コストが低いが、収束が遅いことがある。  | `torch.optim.SGD`             |
| SGD + Momentum          | SGD の改良版。前回の更新方向を加味して振動を抑え、安定した収束を実現。 | `torch.optim.SGD`（`momentum` オプションを使用） |
| AdaGrad           | パラメータごとに学習率を調整。学習率が小さくなりすぎることが課題。      | `torch.optim.Adagrad`         |
| RMSprop           | AdaGrad の学習率低下問題を解決。適応的な学習率を維持し、安定した学習。  | `torch.optim.RMSprop`         |
| Adam              | Momentum と RMSprop を組み合わせたもの。高速収束と安定性が特徴。       | `torch.optim.Adam`            |
